# Understanding Weight duplicating at Scale (Using loop)
Same set of weights (i.e. CNN kernal or filter) to be used by every stride. Here we understand how to reuse the same set of weights as amny times as we like.

In [1]:
import numpy as np
import tensorflow as tf

# Create set of common weights (a CNN filter)

In [2]:
tf.reset_default_graph()

In [3]:
initial_weights = np.array([[1,2,3],[4,5,6],[7,8,9]])
initial_weights

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [4]:
cnn_filter1 = tf.Variable(initial_weights, dtype=tf.float32, name="cnn_filter1")
cnn_filter1

<tf.Variable 'cnn_filter1:0' shape=(3, 3) dtype=float32_ref>

In [5]:
cnn_filter1.name

'cnn_filter1:0'

# setup input image that needs to be <font color="red"> morphed </font> into target image

In [6]:
ip_image=np.array([[10,20,30,40,50,60],[70,80,90,100,110,120],[130,140,150,160,170,180.]])
ip_image

array([[ 10.,  20.,  30.,  40.,  50.,  60.],
       [ 70.,  80.,  90., 100., 110., 120.],
       [130., 140., 150., 160., 170., 180.]])

In [7]:
np.random.seed(123)
target_image = 2*ip_image + np.random.randint(0,5, size=[3,6])
target_image

array([[ 22.,  44.,  62.,  81., 103., 122.],
       [143., 161., 181., 200., 221., 241.],
       [260., 280., 301., 323., 344., 360.]])

### Placeholder to hold your input & target image images into DAG

In [8]:
im_tensor1 = tf.placeholder(dtype=tf.float32, name="im_tensor1")
im_tensor2 = tf.placeholder(dtype=tf.float32, name="im_tensor2")

# Share the weights or filter1 among patches of image
Simply using python variables (No big deal) <br>
- <font color="red">Important to note: ***Don't confuse operation node as "variable"***</font><br>

>**DAG becomes very bulky as number of patches increases**

In [9]:
# Build DAG on just one slice.
# During training, iteratively use sliding slices

losses_list = []
s = 1                # Stride step size
tot_patches = int( (ip_image.shape[1] - initial_weights.shape[1])/s + 1 )

for p_idx in range(tot_patches):
    patch = im_tensor1[:, s*p_idx:s*(p_idx+1)]     # Slicing operation
    a_map = patch*cnn_filter1                      # Element wise multiplication op on that slice
    
    loss = tf.square(im_tensor2[:, s*p_idx:s*(p_idx+1)] - a_map)
    losses_list.append(tf.sqrt(tf.reduce_sum(loss)))    # List of all loss ops. Becomes Tensorflow op named "Pack" (shows as Rank_4 in tensorboard)

In [10]:
losses_list

[<tf.Tensor 'Sqrt:0' shape=() dtype=float32>,
 <tf.Tensor 'Sqrt_1:0' shape=() dtype=float32>,
 <tf.Tensor 'Sqrt_2:0' shape=() dtype=float32>,
 <tf.Tensor 'Sqrt_3:0' shape=() dtype=float32>]

In [11]:
# Cost op
cost = tf.reduce_sum(losses_list, name="cost")   # List (even of operations) can be reduce_sum'ed to create new op, namely "cost"

# Optimizer

In [13]:
optimize = tf.train.GradientDescentOptimizer(learning_rate=0.001)
set_to_optimize = optimize.minimize(cost)

# Go ahead and execute

In [14]:
# Graph writer first
writer = tf.summary.FileWriter(logdir="./Graph", graph=tf.get_default_graph())

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### Just one pass of optimization at a time

In [16]:
# check starting cost
sess.run(cost, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})   

6346.377

In [25]:
# Train 100 times
for i in range(100):
    sess.run(set_to_optimize, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})

sess.run(cost, feed_dict={im_tensor1:ip_image, im_tensor2:target_image})

166.91614

In [26]:
# After finishing optimization (multiple runs of above cell), check CNN-Filter
sess.run(cnn_filter1)

array([[2.0700092, 2.0700092, 2.0700092],
       [2.0141253, 2.0141253, 2.0141253],
       [2.013653 , 2.1378007, 2.2616987]], dtype=float32)

In [ ]:
# Remember the start weights?
initial_weights

In [ ]:
# Conclusion:
# All weights came close to 2. This is expected, as morphed image is just twice of original image

In [51]:
sess.close()
writer.close()